In [65]:
# Balancing the dataset
# From (4:144 2:72 3:12 1:12 0:11) to (4:144 2:144 3:144 1:144 0:144)
# For increase the accuracy
# The outputs are include original data and balanced data


import numpy as np
import pandas as pd
import scipy as sp
from matplotlib import pyplot as plt
import os
import sklearn
from google.colab import drive
from scipy.signal import find_peaks
drive.mount('/content/drive')
path = '/content/drive/My Drive/data/'


from imblearn.over_sampling import RandomOverSampler
from collections import Counter


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# first data import

In [57]:
first_data = []
first_data = pd.read_csv('/content/drive/My Drive/data2/REF.CSV', skiprows=24)
first_data_origin = pd.read_csv('/content/drive/My Drive/data2/REF.CSV', skiprows=24)
first_a = float(first_data_origin.columns[0])
first_b = float(first_data_origin.columns[1])
first_data.loc[0] = [first_a, first_b]
first_data.columns=['Wave_length', 'Amplitude']
first_data['label']=1.3
first_np = first_data[['Wave_length', 'Amplitude']].to_numpy()
first_np = np.ravel(first_np, order='F')
first_np = np.array([first_np])

# functions

#### importing first data function

In [58]:
def first_data_import():
  # import the first_data
  # first_data making
  first_data = []
  first_data = pd.read_csv('/content/drive/My Drive/data2/REF.CSV', skiprows=24)
  first_data_origin = pd.read_csv('/content/drive/My Drive/data2/REF.CSV', skiprows=24)
  first_a = float(first_data_origin.columns[0])
  first_b = float(first_data_origin.columns[1])
  first_data.loc[0] = [first_a, first_b]
  first_data.columns=['Wave_length', 'Amplitude']
  first_data['label']=1.3
  first_np = first_data[['Wave_length', 'Amplitude']].to_numpy()
  first_np = np.ravel(first_np, order='F')
  first_np = np.array([first_np])

  return first_data, first_np

#### importing whole data and stack function

In [59]:
def stack_data(path):
  # the array that combined whole data
  stack_np = np.array([[]])
  # extract the folder name (it will be the label)
  folder_list = os.listdir(path)
  # extract each folder name
  for folder_name in folder_list:
    temp_folder_list = os.listdir(path + folder_name)
    # extract each file name
    for file_name in temp_folder_list:
      # Read the data into a Pandas DataFrame
      data = pd.read_csv(path + folder_name + '/' + file_name, skiprows=24)
      data_origin = pd.read_csv(path + folder_name + '/' + file_name, skiprows=24)
      # generate columns name and restore the removed data(the first data)
      a = float(data_origin.columns[0])
      b = float(data_origin.columns[1])
      data.loc[0] = [a, b]
      data.columns=['Wave_length', 'Amplitude']
      # make the pandas data to numpy
      temp = data[['Wave_length', 'Amplitude']].to_numpy()
      # make it flat
      temp = np.ravel(temp, order='F')
      temp = np.array([temp])
      stack_np = np.concatenate((first_np, temp), axis = 1)
  return stack_np

#### removing DC function

In [60]:
# Implementing a simple DC removal function
def remove_dc(signal):
    # Calculate the mean of the signal
    signal_mean = np.mean(signal)
    # Remove the mean from the signal
    dc_removed_signal = signal - signal_mean
    return dc_removed_signal

#### finding peaks function

In [61]:
# Find peaks
def detect_peaks(signal, threshold=None, min_distance=1, prominence=None):
    if threshold is None:
        # Set threshold as a percentage of the maximum peak
        max_peak = np.max(signal)
    peaks, _ = find_peaks(signal, height=threshold, distance=min_distance, prominence=prominence)
    return peaks

#### peak values extracting function

In [62]:
def extract_peaks(X, Y, peak_range):
  # pandas to numpy
  peak_X = X.to_numpy()
  peak_Y = Y.to_numpy()
  # extract the peak value index
  index = list(peak_Y).index(max(peak_Y))

  # temp saving
  extracted_x = []
  extracted_y = []

  # extract the values from highest peak, and surrounding peaks
  for k in range(peak_range):
    # index labeling
    i = k + 1
    p_temp_index = index + i
    m_temp_index = index - i
    # extract y
    extracted_y.append(peak_Y[p_temp_index])
    extracted_y.append(peak_Y[m_temp_index])
    # extract X
    extracted_x.append(peak_X[p_temp_index])
    extracted_x.append(peak_X[m_temp_index])

  # extrack the center(highest) value
  extracted_x.append(peak_X[index])
  extracted_y.append(peak_Y[index])

  return extracted_x, extracted_y

#### final function

In [63]:
def final_function(path, peak_range):
  # extracted peak amplitudes and wavelength array
  extracted_data = []
  # making label
  label = []
  # the array that combined whole data
  stack_np = np.array([[]])
  # extract the folder name (it will be the label)
  folder_list = os.listdir(path)
  # extract each folder name
  for folder_name in folder_list:
    temp_folder_list = os.listdir(path + folder_name)
    # extract each file name
    for file_name in temp_folder_list:
      # making the label
      if float(folder_name) == 1.3:
        label.append(0)
      elif 1.3 <= float(folder_name) < 1.32:
        label.append(1)
      elif 1.32 <= float(folder_name) < 1.38:
        label.append(2)
      elif 1.38 <= float(folder_name) < 1.39:
        label.append(3)
      elif float(folder_name) >= 1.39:
        label.append(4)

      # Read the data into a Pandas DataFrame
      data = pd.read_csv(path + folder_name + '/' + file_name, skiprows=24)
      data_origin = pd.read_csv(path + folder_name + '/' + file_name, skiprows=24)
      # generate columns name and restore the removed data(the first data)
      a = float(data_origin.columns[0])
      b = float(data_origin.columns[1])
      data.loc[0] = [a, b]
      data.columns=['Wave_length', 'Amplitude']

      # DC removed
      dc_removed_Y = remove_dc(data['Amplitude'])

      # dececting peaks
      min_distance = 100  # Set the minimum distance between peaks
      local_maxima_indices = detect_peaks(dc_removed_Y, min_distance=min_distance)
      local_maxima_x = data['Wave_length'][local_maxima_indices]
      local_maxima_y = dc_removed_Y[local_maxima_indices]

      # extract_peaks
      extracted_x, extracted_y = extract_peaks(local_maxima_x, local_maxima_y, peak_range)
      # saving
      temp = []
      temp.extend(extracted_x)
      temp.extend(extracted_y)
      extracted_data.append(temp)

  label_np = np.array(label)
  extracted_np_data = np.array(extracted_data)
  # data = [wavelength, amplitude] -> the number of each parameter is related with the peak range
  return extracted_np_data, label_np

# Over Sampling: Balanced Label vs Original Label

In [64]:
# Load your data and labels
data, label = final_function(path, 2)
print ('Original_Data')
print (data.shape)
print (label)

counter = Counter(label)

# Print the count of each value
for value, count in counter.items():
    print(f"{value}: {count}")

np.save("/content/drive/MyDrive/extracted_data/data.npy", data)
np.save("/content/drive/MyDrive/extracted_data/label.npy", label)

print ('-----------------------')


# Function to balance the dataset using Random Oversampling
def balance_dataset(X, y):
    # Create an instance of RandomOverSampler
    ros = RandomOverSampler(random_state=42)
    # Resample the dataset
    X_resampled, y_resampled = ros.fit_resample(X, y)
    return X_resampled, y_resampled

# Balance the dataset
balanced_data, balanced_label = balance_dataset(data, label)

# Check the shape of the balanced dataset
print ('Balanced_Data')
print (balanced_data.shape)
print (balanced_label)

counter = Counter(balanced_label)

# Print the count of each value
for value, count in counter.items():
    print(f"{value}: {count}")

# Save the balanced dataset as NumPy arrays
np.save("/content/drive/MyDrive/extracted_data/balanced_data.npy", balanced_data)
np.save("/content/drive/MyDrive/extracted_data/balanced_label.npy", balanced_label)


Original_Data
(251, 10)
[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 2 2 2 2 2 2 2 2 2 2 2 2 4 4 4 4 4 4 4 4 4 4 4 4 2 2 2 2
 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 1 1 1 1 1 1 1 1 1 1 1 1 4 4 4 4 4
 4 4 4 4 4 4 4 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 0 0 0]
4: 144
2: 72
3: 12
1: 12
0: 11
-----------------------
Balanced_Data
(720, 10)
[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 2 2 2 2 2 2 2 2 2 2 2 2 4 4 4 4 4 4 4 4 4 4 4 4 2 2 2 2
 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 1 1 1 1 1 1 1 1 1 1 1 1 4 4 4 4 4
 4 4 4 4 4 4